### boor-system

In [2]:
import sys
sys.path.append('/Users/baza/dev/python-boorstat/')

# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# import plotly.plotly as py
# import plotly.graph_objs as go

import boorstat

### boor-title

Russian Writers CSV

### boor-short-content

[empty]

### boor-full-content

It was always interesting for me how different russian writers relate to each other.<br/>
For example, could Dostoyevky meet Gogol?<br/>
We are going to create csv with brief info like year of life for all russian writers.<br/>
I did not think too much and just parse <a href="https://en.wikipedia.org/wiki/List_of_Russian-language_writers">appropriate wiki page</a> – thanks to it:

In [12]:
import requests
from bs4 import BeautifulSoup

WIKI_RUS_WRITERS_URL = 'https://en.wikipedia.org/wiki/List_of_Russian-language_writers'

soup = BeautifulSoup(requests.get(WIKI_RUS_WRITERS_URL).content, 'lxml')

In [71]:
import re

all_lis = soup.find('div', id='mw-content-text').find_all('li')
lis = [li.text for li in all_lis if re.search(r'> \(', str(li))]

def extract_fields(li):
    name = li.partition('(')[0].strip()

    years = re.search(r'\((.+?\))', li).group(0)
    years = re.findall(r'[0-9]{4}', years)

    birth, death = '', ''
    if len(years) == 1:
        birth = years[0]
    elif len(years) == 2:
        birth, death = years
        
    descr = li.partition(')')[2]
    descr_parts = re.split(r', (?=[A-Z])', descr)
    descr = descr_parts[0].strip().strip(',.;').strip()

    works = ''
    if len(descr_parts) > 1:
        works = ','.join(descr_parts[1:])

    return name, birth, death, descr, works

lis = [extract_fields(li) for li in lis]
lis = [li for li in lis if not (li[1] == '' and li[2] == '')]

columns = ('name', 'birth_year', 'death_year', 'about', 'works')
separator = ';'

def save_to(s):
    s.write(';'.join(columns))
    s.write('\n')
    for li in lis:
        s.write(';'.join(li))
        s.write('\n')
        
from io import StringIO
csv = StringIO()
save_to(csv)

# # or local file
# with open('russian-writers.csv', 'w') as f:
#     save_to(f)

print(csv.getvalue()[:1000], '...')

name;birth_year;death_year;about;works
Alexander Ablesimov;1742;1783;opera librettist, poet, dramatist, satirist and journalist;
Fyodor Abramov;1920;1983;novelist and short story writer;Two Winters and Three Summers
Grigory Adamov;1886;1945;science fiction writer;The Mystery of the Two Oceans
Georgy Adamovich;1892;1972;poet, critic, memoirist, tanslator;
Alexander Afanasyev;1826;1871;folklorist who recorded and published over 600 Russian folktales and fairytales;Russian Fairy Tales
Alexander Afanasyev-Chuzhbinsky;1816;1875;poet, writer, ethnographer and translator;
Alexander Afinogenov;1904;1941;playwright;A Far Place
M. Ageyev;1898;1973;pseudonymous writer;Cocain Romance
Chinghiz Aitmatov;1928;2008;;Kyrgyz novelist and short story writer,Jamilya,The Day Lasts More Than a Hundred Years
David Aizman;1869;1922;;Russian-Jewish writer and playwright
Bella Akhmadulina;1937;2010;modern poet;The String
Anna Akhmatova;1889;1966;acmeist poet;Requiem
Ivan Aksakov;1823;1886;journalist, slavophile

In [3]:
from io import StringIO

import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

Getting csv as pandas dataframe:

In [22]:
df = pd.DataFrame.from_csv(StringIO(requests.get(DUELS_CSV_URL).text), index_col=None)
df.head()

,year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
0,1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlf...,0,0
1,1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0
2,1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high soc...,0,0
3,1819,Wilhelm Kiichelbecker,friend,funny verses about Küchelbecker,0,1
4,1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s ser...,0,0


Looks very exciting as for me:)<br/>
Time to plot Pushkin's Duels Histogram.

In [38]:
life_years = {'start': 1799, 'end': 1837, 'size': 1}

nobody_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 0)]['year'],
    xbins=life_years,
    name='Nobody'
)

only_opponent_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 0) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Only opponent'
)

both_shot_data = go.Histogram(
    x=df[(df['pushkin_shot'] == 1) & (df['opponent_shot'] == 1)]['year'],
    xbins=life_years,
    name='Both shot'
)

data = [only_opponent_shot_data, nobody_shot_data, both_shot_data]
layout = go.Layout(barmode='stack', title="Pushkin's Duels Histogram")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='pushkin-duels')